In [ ]:
import pprint 
import psycopg2 
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT 
from psycopg2.extensions import AsIs 
import os

def pg_con(): 
    con = psycopg2.connect(
            dbname='postgres', 
            user='admin', 
            host='localhost', 
            port='30960', 
            password='password'
    )
    return(con) 

In [1]:
import pprint 
import psycopg2 
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT 
from psycopg2.extensions import AsIs 
import os

con = psycopg2.connect(
            dbname='postgres', 
            user='admin', 
            host='localhost', 
            port='30960', 
            password='password'    
)


In [ ]:
pg_con()

In [ ]:
c=pg_con().cursor() 

In [2]:
con

<connection object at 0x106835828; dsn: 'user=admin password=xxx dbname=postgres host=localhost port=30960', closed: 0>

In [3]:
cur=con.cursor()
cur

<cursor object at 0x10657edd8; closed: 0>

In [ ]:
try:
    c.execute("CREATE TABLE IF NOT EXISTS test2 (id integer, qa text);")
except:
    c.execute("rollback")
    c.execute("CREATE TABLE IF NOT EXISTS test2 (id integer, qa text);")

In [ ]:
c.execute("INSERT INTO test2 VALUES(1,'abc')")

In [ ]:
c.execute("select * from test2")
c.fetchall()

In [4]:
import pandas as pd
df = pd.read_csv('/Users/jasminekaur/Desktop/basf/Code/data/Boston_Crime_Dataset.csv',encoding="latin_1",dtype='unicode',na_values=' ' )  

In [5]:
df.dtypes

INCIDENT_NUMBER        object
OFFENSE_CODE           object
OFFENSE_CODE_GROUP     object
OFFENSE_DESCRIPTION    object
DISTRICT               object
REPORTING_AREA         object
OCCURRED_ON_DATE       object
YEAR                   object
MONTH                  object
DAY_OF_WEEK            object
HOUR                   object
UCR_PART               object
STREET                 object
Lat                    object
Long                   object
Location               object
dtype: object

In [6]:
df = df.drop(columns=['OCCURRED_ON_DATE','Lat', 'Long','Location'])

In [7]:
df.dtypes

INCIDENT_NUMBER        object
OFFENSE_CODE           object
OFFENSE_CODE_GROUP     object
OFFENSE_DESCRIPTION    object
DISTRICT               object
REPORTING_AREA         object
YEAR                   object
MONTH                  object
DAY_OF_WEEK            object
HOUR                   object
UCR_PART               object
STREET                 object
dtype: object

In [8]:
df=df.dropna()

In [9]:
df=df.head(50)

In [10]:
df_OFFENSE = df[['OFFENSE_CODE', 'OFFENSE_CODE_GROUP', 'OFFENSE_DESCRIPTION']].dropna()
df_OFFENSE['OFFENSE_DETAILS']=df_OFFENSE['OFFENSE_CODE']+'_'+df_OFFENSE['OFFENSE_CODE_GROUP']+'_'+df_OFFENSE['OFFENSE_DESCRIPTION']
df_OFFENSE['OFFENSE_DETAILS'] = df_OFFENSE['OFFENSE_DETAILS'].str.replace(' ', '')

In [11]:
df_OFFENSE.nunique()

OFFENSE_CODE           24
OFFENSE_CODE_GROUP     22
OFFENSE_DESCRIPTION    24
OFFENSE_DETAILS        25
dtype: int64

In [12]:
df_OFFENSE = df_OFFENSE.drop_duplicates(subset='OFFENSE_DETAILS', keep='first') #.reset_index()
df_OFFENSE['OFFENSE_ID']=range(1, len(df_OFFENSE.index)+1)
df_OFFENSE = df_OFFENSE[['OFFENSE_ID','OFFENSE_CODE', 'OFFENSE_CODE_GROUP', 'OFFENSE_DESCRIPTION','OFFENSE_DETAILS']]
df_OFFENSE=df_OFFENSE.astype('str')

In [13]:
df_OFFENSE.isnull().sum()

OFFENSE_ID             0
OFFENSE_CODE           0
OFFENSE_CODE_GROUP     0
OFFENSE_DESCRIPTION    0
OFFENSE_DETAILS        0
dtype: int64

In [14]:
df_OFFENSE.head()

,OFFENSE_ID,OFFENSE_CODE,OFFENSE_CODE_GROUP,OFFENSE_DESCRIPTION,OFFENSE_DETAILS
0,1,2403,Disorderly Conduct,DISTURBING THE PEACE,2403_DisorderlyConduct_DISTURBINGTHEPEACE
1,2,3201,Property Lost,PROPERTY - LOST,3201_PropertyLost_PROPERTY-LOST
2,3,2647,Other,THREATS TO DO BODILY HARM,2647_Other_THREATSTODOBODILYHARM
3,4,413,Aggravated Assault,ASSAULT - AGGRAVATED - BATTERY,413_AggravatedAssault_ASSAULT-AGGRAVATED-BATTERY
4,5,3122,Aircraft,AIRCRAFT INCIDENTS,3122_Aircraft_AIRCRAFTINCIDENTS


In [15]:
cur.execute("DROP TABLE IF EXISTS OFFENSE");

In [16]:
cur.execute("""CREATE TABLE OFFENSE
(OFFENSE_ID int NOT NULL PRIMARY KEY,
OFFENSE_CODE integer NULL,
OFFENSE_CODE_GROUP text NULL,
OFFENSE_DESCRIPTION text NULL,
OFFENSE_DETAILS text NULL);""")

In [17]:
Offense_Cols = ",".join([str(i) for i in df_OFFENSE.columns.tolist()])

In [18]:
Offense_Cols

'OFFENSE_ID,OFFENSE_CODE,OFFENSE_CODE_GROUP,OFFENSE_DESCRIPTION,OFFENSE_DETAILS'

In [19]:
for i, row in df_OFFENSE.iterrows():
        sql = "INSERT INTO OFFENSE (" +Offense_Cols+ ") VALUES (" + "%s,"*(len(row)-1) + "%s);"
        cur.execute(sql,tuple(row))
        con.commit()

In [20]:
cur.execute("SELECT * FROM OFFENSE")
cur.fetchall()

[(1,
  2403,
  'Disorderly Conduct',
  'DISTURBING THE PEACE',
  '2403_DisorderlyConduct_DISTURBINGTHEPEACE'),
 (2,
  3201,
  'Property Lost',
  'PROPERTY - LOST',
  '3201_PropertyLost_PROPERTY-LOST'),
 (3,
  2647,
  'Other',
  'THREATS TO DO BODILY HARM',
  '2647_Other_THREATSTODOBODILYHARM'),
 (4,
  413,
  'Aggravated Assault',
  'ASSAULT - AGGRAVATED - BATTERY',
  '413_AggravatedAssault_ASSAULT-AGGRAVATED-BATTERY'),
 (5,
  3122,
  'Aircraft',
  'AIRCRAFT INCIDENTS',
  '3122_Aircraft_AIRCRAFTINCIDENTS'),
 (6, 1402, 'Vandalism', 'VANDALISM', '1402_Vandalism_VANDALISM'),
 (7,
  3301,
  'Verbal Disputes',
  'VERBAL DISPUTE',
  '3301_VerbalDisputes_VERBALDISPUTE'),
 (8,
  802,
  'Simple Assault',
  'ASSAULT SIMPLE - BATTERY',
  '802_SimpleAssault_ASSAULTSIMPLE-BATTERY'),
 (9, 3410, 'Towed', 'TOWED MOTOR VEHICLE', '3410_Towed_TOWEDMOTORVEHICLE'),
 (10,
  3803,
  'Motor Vehicle Accident Response',
  'M/V ACCIDENT - PERSONAL INJURY',
  '3803_MotorVehicleAccidentResponse_M/VACCIDENT-PERSONAL

In [21]:
cur.execute("SELECT * FROM OFFENSE WHERE OFFENSE_DESCRIPTION ILIKE 'pro%'")
cur.fetchall()

[(2,
  3201,
  'Property Lost',
  'PROPERTY - LOST',
  '3201_PropertyLost_PROPERTY-LOST')]

In [22]:
cur.execute("DROP TABLE IF EXISTS INCIDENT")

In [23]:
cur.execute("""CREATE TABLE INCIDENT(
    INCIDENT_NUMBER text,
    OFFENSE_CODE integer,
    OFFENSE_CODE_GROUP text,
    OFFENSE_DESCRIPTION text,
    DISTRICT text, 
    REPORTING_AREA integer,
    YEAR integer,
    MONTH text,
    DAY_OF_WEEK text,
    HOUR integer,
    UCR_PART text,
    STREET text);""")
con.commit()

In [24]:
Incident_Cols = ",".join([str(i) for i in df.columns.tolist()])
Incident_Cols

'INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_CODE_GROUP,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,YEAR,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,STREET'

In [25]:
for i, row in df.iterrows():
        sql = "INSERT INTO INCIDENT (" +Incident_Cols + ") VALUES (" + "%s,"*(len(row)-1) + "%s);"
        cur.execute(sql,tuple(row))
        con.commit()

In [26]:
cur.execute("SELECT * FROM INCIDENT")
cur.fetchall()

[('I182080058',
  2403,
  'Disorderly Conduct',
  'DISTURBING THE PEACE',
  'E18',
  495,
  2018,
  '10',
  'Wednesday',
  20,
  'Part Two',
  'ARLINGTON ST'),
 ('I182080053',
  3201,
  'Property Lost',
  'PROPERTY - LOST',
  'D14',
  795,
  2018,
  '8',
  'Thursday',
  20,
  'Part Three',
  'ALLSTON ST'),
 ('I182080052',
  2647,
  'Other',
  'THREATS TO DO BODILY HARM',
  'B2',
  329,
  2018,
  '10',
  'Wednesday',
  19,
  'Part Two',
  'DEVON ST'),
 ('I182080051',
  413,
  'Aggravated Assault',
  'ASSAULT - AGGRAVATED - BATTERY',
  'A1',
  92,
  2018,
  '10',
  'Wednesday',
  20,
  'Part One',
  'CAMBRIDGE ST'),
 ('I182080050',
  3122,
  'Aircraft',
  'AIRCRAFT INCIDENTS',
  'A7',
  36,
  2018,
  '10',
  'WED',
  20,
  'Part Three',
  'PRESCOTT ST'),
 ('I182080049',
  1402,
  'Vandalism',
  'VANDALISM',
  'C11',
  351,
  2018,
  '10',
  'Tuesday',
  20,
  'Part Two',
  'DORCHESTER AVE'),
 ('I182080047',
  3301,
  'Verbal Disputes',
  'VERBAL DISPUTE',
  'B2',
  603,
  2018,
  '10',
 

In [27]:
cur.execute("SELECT * FROM INCIDENT WHERE OFFENSE_DESCRIPTION ILIKE 'pro%'")
cur.fetchall()

[('I182080053',
  3201,
  'Property Lost',
  'PROPERTY - LOST',
  'D14',
  795,
  2018,
  '8',
  'Thursday',
  20,
  'Part Three',
  'ALLSTON ST'),
 ('I182079996',
  3201,
  'Property Lost',
  'PROPERTY - LOST',
  'B3',
  922,
  2018,
  '10',
  'MONDAY',
  7,
  'Part Three',
  'DR. MARY MOORE BEATTY CIR')]